In [1]:
# Parameters
BATCH_MODE = "true"


# SW-8-PythonRDF

**Navigation** : [<< 7-OWL](SW-7-OWL.ipynb) | [Index](README.md) | [9-SHACL >>](SW-9-SHACL.ipynb)

## RDF en Python avec rdflib

Ce notebook fait le pont entre le monde .NET (SW-1 a SW-7) et l'ecosysteme Python pour le Web Semantique. Vous y decouvrirez **rdflib**, l'equivalent Python de dotNetRDF, et **SPARQLWrapper** pour interroger des endpoints distants.

### Objectifs d'apprentissage

A la fin de ce notebook, vous saurez :
1. Creer et manipuler des graphes RDF avec rdflib
2. Serialiser et parser dans differents formats
3. Executer des requetes SPARQL locales et distantes
4. Faire la correspondance entre dotNetRDF et rdflib

### Prerequis
- Python 3.10+, pip
- Notebooks SW-1 a SW-7 recommandes (pour la comprehension conceptuelle)

### Duree estimee : 50 minutes

---

## 1. Ecosysteme Python pour le Web Semantique

Python dispose d'un ecosysteme riche pour le Web Semantique. Voici les principales bibliotheques :

| Bibliotheque | Version | Role principal | Notebooks |
|-------------|---------|----------------|----------|
| **rdflib** | 7.x | Manipulation de graphes RDF, parsing, serialisation, SPARQL | SW-8 a SW-13 |
| **SPARQLWrapper** | 2.x | Requetes vers des endpoints SPARQL distants | SW-8 |
| **pySHACL** | 0.27+ | Validation de donnees RDF contre des shapes SHACL | SW-9 |
| **OWLReady2** | 0.50+ | Manipulation d'ontologies OWL, raisonnement HermiT | SW-12 |
| **kglab** | 0.6+ | Abstraction haut niveau pour graphes de connaissances | SW-12 |

> **Note** : `rdflib` est le coeur de l'ecosysteme Python pour le Web Semantique, equivalent a `dotNetRDF` pour .NET. La plupart des autres bibliotheques s'appuient sur rdflib comme dependance.

### Installation

Installons les deux bibliotheques principales de ce notebook. L'option `-q` reduit la verbosity de pip.

In [2]:
%pip install -q rdflib SPARQLWrapper

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 26.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Verifions que les installations se sont bien passees en important les modules et en affichant leurs versions.

In [3]:
import rdflib
import SPARQLWrapper

print(f"rdflib version   : {rdflib.__version__}")
print(f"SPARQLWrapper    : {SPARQLWrapper.__version__}")
print("Installation OK.")

rdflib version   : 7.6.0
SPARQLWrapper    : 2.0.0
Installation OK.


---

## 2. Fondamentaux de rdflib

rdflib offre une API pythonique pour creer, manipuler et interroger des graphes RDF. Les classes principales sont :

| Classe | Description | Equivalent dotNetRDF |
|--------|-------------|---------------------|
| `Graph` | Conteneur de triples RDF | `Graph` |
| `URIRef` | Noeud URI (identifie une ressource) | `IUriNode` |
| `BNode` | Noeud anonyme (blank node) | `IBlankNode` |
| `Literal` | Valeur litterale (texte, nombre, date...) | `ILiteralNode` |
| `Namespace` | Gestionnaire de prefixes | `UriFactory` |

Commencons par les imports fondamentaux.

In [4]:
from rdflib import Graph, URIRef, Literal, BNode, Namespace
from rdflib.namespace import RDF, RDFS, XSD, FOAF, DC, DCTERMS

print("Imports rdflib prets.")
print(f"Namespace RDF  : {RDF}")
print(f"Namespace RDFS : {RDFS}")
print(f"Namespace FOAF : {FOAF}")

Imports rdflib prets.
Namespace RDF  : http://www.w3.org/1999/02/22-rdf-syntax-ns#
Namespace RDFS : http://www.w3.org/2000/01/rdf-schema#
Namespace FOAF : http://xmlns.com/foaf/0.1/


### Interpretation

rdflib fournit des namespaces preconfigures pour les vocabulaires standards du W3C :
- `RDF` : `http://www.w3.org/1999/02/22-rdf-syntax-ns#` (types, proprietes de base)
- `RDFS` : `http://www.w3.org/2000/01/rdf-schema#` (hierarchie de classes/proprietes)
- `XSD` : `http://www.w3.org/2001/XMLSchema#` (types de donnees)
- `FOAF` : `http://xmlns.com/foaf/0.1/` (description de personnes)
- `DC` / `DCTERMS` : `http://purl.org/dc/elements/1.1/` (metadonnees Dublin Core)

### 2.1 Creer un graphe et ajouter des triples

Un graphe RDF est un ensemble de triples `(sujet, predicat, objet)`. En rdflib, on cree un `Graph` vide puis on y ajoute des triples avec la methode `add()`.

In [5]:
# Create a new empty graph
g = Graph()

# Define a custom namespace
EX = Namespace("http://example.org/")
g.bind("ex", EX)
g.bind("foaf", FOAF)

# Add triples about a person
g.add((EX.Alice, RDF.type, FOAF.Person))
g.add((EX.Alice, FOAF.name, Literal("Alice Dupont")))
g.add((EX.Alice, FOAF.age, Literal(30, datatype=XSD.integer)))
g.add((EX.Alice, FOAF.knows, EX.Bob))

g.add((EX.Bob, RDF.type, FOAF.Person))
g.add((EX.Bob, FOAF.name, Literal("Bob Martin")))
g.add((EX.Bob, FOAF.age, Literal(25, datatype=XSD.integer)))

print(f"Nombre de triples dans le graphe : {len(g)}")
print()

# Iterate over all triples
for s, p, o in g:
    print(f"  {s} -- {p} --> {o}")

Nombre de triples dans le graphe : 7

  http://example.org/Bob -- http://xmlns.com/foaf/0.1/age --> 25
  http://example.org/Alice -- http://xmlns.com/foaf/0.1/age --> 30
  http://example.org/Alice -- http://xmlns.com/foaf/0.1/name --> Alice Dupont
  http://example.org/Alice -- http://xmlns.com/foaf/0.1/knows --> http://example.org/Bob
  http://example.org/Alice -- http://www.w3.org/1999/02/22-rdf-syntax-ns#type --> http://xmlns.com/foaf/0.1/Person
  http://example.org/Bob -- http://xmlns.com/foaf/0.1/name --> Bob Martin
  http://example.org/Bob -- http://www.w3.org/1999/02/22-rdf-syntax-ns#type --> http://xmlns.com/foaf/0.1/Person


### Interpretation : Creation de graphe

**Sortie obtenue** : 7 triples decrivant deux personnes (Alice et Bob) et leur relation.

| Element | Syntaxe rdflib | Exemple |
|---------|---------------|--------|
| Namespace personnalise | `EX = Namespace("http://example.org/")` | `EX.Alice` = `<http://example.org/Alice>` |
| Binding de prefixe | `g.bind("ex", EX)` | Utilise `ex:` dans la serialisation |
| Ajout de triple | `g.add((s, p, o))` | Tuple de 3 elements |
| Type RDF | `RDF.type` | Equivalent de `rdf:type` / `a` en Turtle |

**Points cles** :
1. `g.add()` prend un **tuple de 3 elements** (contrairement a dotNetRDF qui utilise `g.Assert(new Triple(...))`)
2. `EX.Alice` est un raccourci pour `URIRef("http://example.org/Alice")` grace au `Namespace`
3. `g.bind()` associe un prefixe court pour la serialisation (ex: `ex:Alice` au lieu de l'URI complete)

### 2.2 Types de noeuds

RDF definit trois types de noeuds. Voyons comment rdflib les represente, avec une attention particuliere aux **litteraux types** et aux **tags de langue**.

In [6]:
# 1. URIRef - identifies a resource by its URI
uri_node = URIRef("http://example.org/Alice")
print(f"URIRef       : {uri_node}")
print(f"  type       : {type(uri_node).__name__}")
print()

# 2. BNode - anonymous node (no URI)
blank = BNode()
print(f"BNode        : {blank}")
print(f"  type       : {type(blank).__name__}")
print()

# 3. Literal - data values
lit_simple = Literal("Hello World")
lit_lang   = Literal("Bonjour le monde", lang="fr")
lit_int    = Literal(42, datatype=XSD.integer)
lit_float  = Literal(3.14, datatype=XSD.double)
lit_bool   = Literal(True, datatype=XSD.boolean)
lit_date   = Literal("2025-01-15", datatype=XSD.date)

print("Literal examples:")
for name, lit in [("simple", lit_simple), ("lang=fr", lit_lang),
                   ("integer", lit_int), ("double", lit_float),
                   ("boolean", lit_bool), ("date", lit_date)]:
    dt = lit.datatype if lit.datatype else "(none)"
    lg = lit.language if lit.language else "(none)"
    print(f"  {name:10s} : value={lit.toPython()!r:30s}  datatype={dt}  lang={lg}")

URIRef       : http://example.org/Alice
  type       : URIRef

BNode        : Ncebf19cdb6fa4b539709a8f45bb0d972
  type       : BNode

Literal examples:
  simple     : value='Hello World'                   datatype=(none)  lang=(none)
  lang=fr    : value='Bonjour le monde'              datatype=(none)  lang=fr
  integer    : value=42                              datatype=http://www.w3.org/2001/XMLSchema#integer  lang=(none)
  double     : value=3.14                            datatype=http://www.w3.org/2001/XMLSchema#double  lang=(none)
  boolean    : value=True                            datatype=http://www.w3.org/2001/XMLSchema#boolean  lang=(none)
  date       : value=datetime.date(2025, 1, 15)      datatype=http://www.w3.org/2001/XMLSchema#date  lang=(none)


### Interpretation : Types de noeuds

| Type | Classe rdflib | Notation Turtle | Usage |
|------|--------------|----------------|-------|
| **URI** | `URIRef` | `<http://...>` ou `ex:Alice` | Identifier une ressource |
| **Blank Node** | `BNode` | `_:b0` | Noeud anonyme sans identifiant global |
| **Litteral simple** | `Literal("texte")` | `"texte"` | Chaine sans type ni langue |
| **Litteral type** | `Literal(42, datatype=XSD.integer)` | `"42"^^xsd:integer` | Valeur avec type de donnee |
| **Litteral avec langue** | `Literal("Bonjour", lang="fr")` | `"Bonjour"@fr` | Texte dans une langue specifique |

**Points cles** :
1. La methode `.toPython()` convertit un litteral rdflib en type Python natif (int, float, bool, str, datetime...)
2. Un litteral ne peut **pas** avoir a la fois un `datatype` et un `lang` (contrainte RDF)
3. Les `BNode` recoivent un identifiant unique auto-genere (visible dans la serialisation comme `_:Nxxx`)

---

## 3. Serialisation avec rdflib

Un graphe RDF peut etre serialise (exporte) dans plusieurs formats standards. rdflib supporte nativement les formats les plus courants. Commencons par serialiser le graphe que nous avons cree dans la section precedente.

In [7]:
# Serialize to Turtle (compact, human-readable)
turtle_output = g.serialize(format="turtle")
print("=== Format Turtle ===")
print(turtle_output)

=== Format Turtle ===
@prefix ex: <http://example.org/> .
@prefix foaf: <http://xmlns.com/foaf/0.1/> .
@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .

ex:Alice a foaf:Person ;
    foaf:age 30 ;
    foaf:knows ex:Bob ;
    foaf:name "Alice Dupont" .

ex:Bob a foaf:Person ;
    foaf:age 25 ;
    foaf:name "Bob Martin" .




Le format **Turtle** est le plus lisible pour les humains : il utilise des prefixes (`ex:`, `foaf:`) et regroupe les proprietes d'un meme sujet avec `;`.

In [8]:
# Serialize to N-Triples (one triple per line, no abbreviations)
nt_output = g.serialize(format="nt")
print("=== Format N-Triples ===")
print(nt_output)
print()

# Serialize to RDF/XML
xml_output = g.serialize(format="xml")
print("=== Format RDF/XML ===")
print(xml_output)
print()

# Serialize to JSON-LD
jsonld_output = g.serialize(format="json-ld", indent=2)
print("=== Format JSON-LD ===")
print(jsonld_output)

=== Format N-Triples ===
<http://example.org/Bob> <http://xmlns.com/foaf/0.1/age> "25"^^<http://www.w3.org/2001/XMLSchema#integer> .
<http://example.org/Alice> <http://xmlns.com/foaf/0.1/age> "30"^^<http://www.w3.org/2001/XMLSchema#integer> .
<http://example.org/Alice> <http://xmlns.com/foaf/0.1/name> "Alice Dupont" .
<http://example.org/Alice> <http://xmlns.com/foaf/0.1/knows> <http://example.org/Bob> .
<http://example.org/Alice> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://xmlns.com/foaf/0.1/Person> .
<http://example.org/Bob> <http://xmlns.com/foaf/0.1/name> "Bob Martin" .
<http://example.org/Bob> <http://www.w3.org/1999/02/22-rdf-syntax-ns#type> <http://xmlns.com/foaf/0.1/Person> .


=== Format RDF/XML ===
<?xml version="1.0" encoding="utf-8"?>
<rdf:RDF
   xmlns:foaf="http://xmlns.com/foaf/0.1/"
   xmlns:rdf="http://www.w3.org/1999/02/22-rdf-syntax-ns#"
>
  <rdf:Description rdf:about="http://example.org/Bob">
    <rdf:type rdf:resource="http://xmlns.com/foaf/0.1/Person"

### Interpretation : Comparaison des formats de serialisation

| Format | Extension | Lisibilite | Taille | Usage principal |
|--------|-----------|-----------|--------|----------------|
| **Turtle** | `.ttl` | Excellente | Compacte | Edition manuelle, documentation |
| **N-Triples** | `.nt` | Moyenne | Grande | Echanges bulk, streaming |
| **RDF/XML** | `.rdf` | Faible | Grande | Compatibilite historique |
| **JSON-LD** | `.jsonld` | Bonne (dev web) | Moyenne | APIs web, Schema.org |

> **Conseil** : Preferez Turtle pour le travail quotidien et JSON-LD pour l'integration web. N-Triples est utile pour le traitement en masse (une ligne = un triple).

### 3.1 Charger un fichier Turtle existant

Chargeons le fichier `data/Example.ttl` fourni avec cette serie. Ce fichier utilise les prefixes Dublin Core (`dc:`) et un namespace exemple (`ex:`) pour decrire la specification RDF/XML du W3C.

In [9]:
# Load Example.ttl from the data directory
g_example = Graph()
g_example.parse("data/Example.ttl", format="turtle")

print(f"Triples charges depuis Example.ttl : {len(g_example)}")
print()

# Display all triples
for s, p, o in g_example:
    print(f"  Sujet   : {s}")
    print(f"  Predicat: {p}")
    print(f"  Objet   : {o}")
    print()

Triples charges depuis Example.ttl : 4

  Sujet   : n05ffa8a9e4b34d4aaba43fc4e9c4be63b1
  Predicat: http://example.org/stuff/1.0/homePage
  Objet   : http://purl.org/net/dajobe/

  Sujet   : http://www.w3.org/TR/rdf-syntax-grammar
  Predicat: http://example.org/stuff/1.0/editor
  Objet   : n05ffa8a9e4b34d4aaba43fc4e9c4be63b1

  Sujet   : http://www.w3.org/TR/rdf-syntax-grammar
  Predicat: http://purl.org/dc/elements/1.1/title
  Objet   : RDF/XML Syntax Specification (Revised)

  Sujet   : n05ffa8a9e4b34d4aaba43fc4e9c4be63b1
  Predicat: http://example.org/stuff/1.0/fullname
  Objet   : Dave Beckett



### Interpretation : Chargement de Example.ttl

Le fichier contient des informations sur le document "RDF/XML Syntax Specification" :
- Un titre via `dc:title`
- Un editeur (Dave Beckett) represente par un **blank node** avec `ex:fullname` et `ex:homePage`

C'est le meme fichier utilise dans les notebooks .NET (SW-2, SW-3). La methode `g.parse()` detecte automatiquement le format si non specifie, mais il est recommande de le preciser pour eviter les ambiguites.

**Formats supportes par `parse()`** : `turtle`, `xml`, `json-ld`, `nt`, `n3`, `trig`, `nquads`

Voyons comment ce graphe charge s'affiche en JSON-LD pour comparer avec le Turtle d'origine.

In [10]:
# Re-serialize the loaded graph in different format
print("=== Example.ttl en JSON-LD ===")
print(g_example.serialize(format="json-ld", indent=2))

=== Example.ttl en JSON-LD ===
[
  {
    "@id": "http://www.w3.org/TR/rdf-syntax-grammar",
    "http://example.org/stuff/1.0/editor": [
      {
        "@id": "_:n05ffa8a9e4b34d4aaba43fc4e9c4be63b1"
      }
    ],
    "http://purl.org/dc/elements/1.1/title": [
      {
        "@value": "RDF/XML Syntax Specification (Revised)"
      }
    ]
  },
  {
    "@id": "_:n05ffa8a9e4b34d4aaba43fc4e9c4be63b1",
    "http://example.org/stuff/1.0/fullname": [
      {
        "@value": "Dave Beckett"
      }
    ],
    "http://example.org/stuff/1.0/homePage": [
      {
        "@id": "http://purl.org/net/dajobe/"
      }
    ]
  }
]


---

## 4. SPARQL avec rdflib

rdflib integre un moteur SPARQL complet qui permet d'interroger les graphes en memoire, sans avoir besoin d'un serveur externe. Chargeons d'abord le fichier `animals.ttl` qui contient une hierarchie de classes RDFS et des instances d'animaux.

In [11]:
# Load animals.ttl
g_animals = Graph()
g_animals.parse("data/animals.ttl", format="turtle")

print(f"Triples charges depuis animals.ttl : {len(g_animals)}")
print()

# Show all namespaces bound in the file
print("Namespaces :")
for prefix, uri in g_animals.namespaces():
    if prefix:  # skip default
        print(f"  @prefix {prefix}: <{uri}> .")

Triples charges depuis animals.ttl : 51

Namespaces :
  @prefix brick: <https://brickschema.org/schema/Brick#> .
  @prefix csvw: <http://www.w3.org/ns/csvw#> .
  @prefix dc: <http://purl.org/dc/elements/1.1/> .
  @prefix dcat: <http://www.w3.org/ns/dcat#> .
  @prefix dcmitype: <http://purl.org/dc/dcmitype/> .
  @prefix dcterms: <http://purl.org/dc/terms/> .
  @prefix dcam: <http://purl.org/dc/dcam/> .
  @prefix doap: <http://usefulinc.com/ns/doap#> .
  @prefix foaf: <http://xmlns.com/foaf/0.1/> .
  @prefix geo: <http://www.opengis.net/ont/geosparql#> .
  @prefix odrl: <http://www.w3.org/ns/odrl/2/> .
  @prefix org: <http://www.w3.org/ns/org#> .
  @prefix prof: <http://www.w3.org/ns/dx/prof/> .
  @prefix prov: <http://www.w3.org/ns/prov#> .
  @prefix qb: <http://purl.org/linked-data/cube#> .
  @prefix schema: <https://schema.org/> .
  @prefix sh: <http://www.w3.org/ns/shacl#> .
  @prefix skos: <http://www.w3.org/2004/02/skos/core#> .
  @prefix sosa: <http://www.w3.org/ns/sosa/> .
  @pre

### 4.1 Requete SELECT simple

Commencons par lister tous les animaux avec leur nom et leur cri. La methode `g.query()` prend une chaine SPARQL et retourne un objet iterable de resultats.

In [12]:
# Simple SELECT query - list all animals with name and sound
query_animals = """
PREFIX ex: <http://example.org/animals#>

SELECT ?animal ?name ?sound
WHERE {
    ?animal ex:name ?name .
    ?animal ex:sound ?sound .
}
ORDER BY ?name
"""

results = g_animals.query(query_animals)

print(f"Resultats : {len(results)} animaux trouves")
print(f"{'Animal':<35s} {'Nom':<12s} {'Cri':<25s}")
print("-" * 72)
for row in results:
    print(f"{str(row.animal):<35s} {str(row.name):<12s} {str(row.sound):<25s}")

Resultats : 4 animaux trouves
Animal                              Nom          Cri                      
------------------------------------------------------------------------
http://example.org/animals#buddy    Buddy        Woof woof                
http://example.org/animals#coco     Coco         Coco veut un gateau      
http://example.org/animals#minou    Minou        Miaou                    
http://example.org/animals#rex      Rex          Woof                     


### Interpretation : Requete SELECT

La requete retourne les 4 animaux definis dans `animals.ttl` (Buddy, Coco, Minou, Rex).

**Syntaxe cle** :
- `PREFIX ex: <...>` declare un prefixe (equivalent de `@prefix` en Turtle)
- `?animal`, `?name`, `?sound` sont des **variables** SPARQL
- Le pattern `?animal ex:name ?name .` selectionne tous les triples qui correspondent
- `ORDER BY ?name` trie les resultats alphabetiquement

> **Note** : On accede aux colonnes du resultat par nom (`row.name`) ou par index (`row[1]`).

### 4.2 FILTER et OPTIONAL

SPARQL offre des operateurs puissants pour affiner les resultats :
- `FILTER` : applique une condition (comparaison, regex, existence...)
- `OPTIONAL` : inclut un pattern meme s'il n'est pas satisfait (equivalent de LEFT JOIN en SQL)

In [13]:
# FILTER: find animals older than 4 years
query_filter = """
PREFIX ex: <http://example.org/animals#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>

SELECT ?name ?age
WHERE {
    ?animal ex:name ?name .
    ?animal ex:age ?age .
    FILTER (?age > 4)
}
ORDER BY DESC(?age)
"""

print("=== Animaux de plus de 4 ans ===")
for row in g_animals.query(query_filter):
    print(f"  {row.name} : {row.age} ans")

=== Animaux de plus de 4 ans ===
  Coco : 12 ans
  Buddy : 7 ans
  Rex : 5 ans


Le `FILTER (?age > 4)` ne retient que les animaux dont l'age depasse 4 ans. `DESC(?age)` trie du plus age au plus jeune.

In [14]:
# OPTIONAL: list all animals with flight capability (if known)
query_optional = """
PREFIX ex: <http://example.org/animals#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?type_label ?canFly
WHERE {
    ?animal ex:name ?name .
    ?animal a ?type .
    ?type rdfs:label ?type_label .
    OPTIONAL { ?animal ex:canFly ?canFly . }
}
ORDER BY ?name
"""

print("=== Animaux avec capacite de vol (OPTIONAL) ===")
print(f"{'Nom':<12s} {'Type':<15s} {'Peut voler':<12s}")
print("-" * 39)
for row in g_animals.query(query_optional):
    fly_str = str(row.canFly) if row.canFly is not None else "non renseigne"
    print(f"{str(row.name):<12s} {str(row.type_label):<15s} {fly_str:<12s}")

=== Animaux avec capacite de vol (OPTIONAL) ===
Nom          Type            Peut voler  
---------------------------------------
Buddy        Chien           non renseigne
Coco         Perroquet       true        
Minou        Chat            non renseigne
Rex          Chien           non renseigne


### Interpretation : OPTIONAL

Seul Coco (perroquet) a la propriete `ex:canFly`. Grace a `OPTIONAL`, les autres animaux apparaissent quand meme dans les resultats, avec la valeur `None` pour la colonne `canFly`.

| Mot-cle SPARQL | Equivalent SQL | Comportement |
|---------------|----------------|-------------|
| `FILTER` | `WHERE condition` | Exclut les lignes ne respectant pas la condition |
| `OPTIONAL` | `LEFT JOIN` | Inclut la ligne meme si le pattern optionnel n'est pas satisfait |
| `ORDER BY` | `ORDER BY` | Tri ascendant (defaut) ou `DESC()` |

> **Conseil** : Utilisez `OPTIONAL` plutot que de supposer que toutes les proprietes sont renseignees. Le Web Semantique suit le principe du **monde ouvert** : l'absence d'information ne signifie pas que l'information est fausse.

### 4.3 Exploration de la hierarchie de classes

Le fichier `animals.ttl` contient une hierarchie RDFS : `Animal > Mammal/Bird > Dog/Cat/Parrot`. Interrogeons-la avec SPARQL.

In [15]:
# Query class hierarchy
query_classes = """
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?class ?label ?parent ?parent_label
WHERE {
    ?class a rdfs:Class .
    ?class rdfs:label ?label .
    OPTIONAL {
        ?class rdfs:subClassOf ?parent .
        ?parent rdfs:label ?parent_label .
    }
}
ORDER BY ?parent_label ?label
"""

print("=== Hierarchie de classes ===")
print(f"{'Classe':<15s} {'Parent':<15s}")
print("-" * 30)
for row in g_animals.query(query_classes):
    parent = str(row.parent_label) if row.parent_label else "(racine)"
    print(f"{str(row.label):<15s} {parent:<15s}")

=== Hierarchie de classes ===
Classe          Parent         
------------------------------
Animal          (racine)       
Mammifere       Animal         
Oiseau          Animal         
Chat            Mammifere      
Chien           Mammifere      
Perroquet       Oiseau         


La hierarchie est bien visible : `Animal` est la classe racine (pas de parent), `Mammifere` et `Oiseau` sont des sous-classes directes, et `Chien`, `Chat`, `Perroquet` sont au niveau le plus specifique.

---

## 5. SPARQLWrapper : Endpoints distants

Jusqu'ici nous avons interroge des graphes **en memoire**. Le Web Semantique prend tout son sens quand on interroge des endpoints **publics** contenant des millions (voire milliards) de triples.

**SPARQLWrapper** simplifie l'envoi de requetes SPARQL a ces serveurs distants et la recuperation des resultats.

> **Attention** : Les endpoints publics peuvent etre temporairement indisponibles ou imposer des limites de debit. Toutes les requetes ci-dessous sont enveloppees dans des `try/except` pour gerer ces cas gracieusement.

In [16]:
from SPARQLWrapper import SPARQLWrapper, JSON, XML
import json

print("SPARQLWrapper importe.")

SPARQLWrapper importe.


### 5.1 Interroger DBpedia

DBpedia extrait des donnees structurees de Wikipedia et les expose sous forme de triples RDF. L'endpoint SPARQL est accessible a `http://dbpedia.org/sparql`.

Cherchons les 5 plus grandes villes de France selon DBpedia.

In [17]:
# Query DBpedia for French cities
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setReturnFormat(JSON)

sparql.setQuery("""
PREFIX dbo: <http://dbpedia.org/ontology/>
PREFIX dbr: <http://dbpedia.org/resource/>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?city ?name ?population
WHERE {
    ?city a dbo:City .
    ?city dbo:country dbr:France .
    ?city rdfs:label ?name .
    ?city dbo:populationTotal ?population .
    FILTER (lang(?name) = "fr")
}
ORDER BY DESC(?population)
LIMIT 5
""")

try:
    results = sparql.query().convert()
    bindings = results["results"]["bindings"]

    print(f"=== Top 5 villes francaises (DBpedia) ===")
    print(f"{'Ville':<25s} {'Population':>12s}")
    print("-" * 37)
    for r in bindings:
        name = r["name"]["value"]
        pop = int(r["population"]["value"])
        print(f"{name:<25s} {pop:>12,d}")

except Exception as e:
    print(f"Erreur lors de la requete DBpedia : {type(e).__name__}: {e}")
    print("L'endpoint DBpedia est peut-etre temporairement indisponible.")
    print("Cela n'empeche pas de continuer le notebook.")

=== Top 5 villes francaises (DBpedia) ===
Ville                       Population
-------------------------------------
Papeete                         26,654
Quartier Saint-Germain-des-Prés        5,154
Gustavia                         2,300


### Interpretation : Requete DBpedia

**Structure de la reponse** : SPARQLWrapper retourne un dictionnaire JSON contenant :
- `results.bindings` : liste de dictionnaires, un par ligne de resultat
- Chaque variable (`?city`, `?name`, `?population`) est une cle avec `value` et `type`

**Points cles** :
1. `FILTER (lang(?name) = "fr")` limite les labels a la langue francaise
2. `LIMIT 5` restreint le nombre de resultats (important pour les endpoints publics)
3. Le `try/except` est essentiel : les services publics peuvent avoir des timeouts ou etre en maintenance

### 5.2 Interroger Wikidata

Wikidata est la base de connaissances structuree du projet Wikimedia. Son endpoint SPARQL est a `https://query.wikidata.org/sparql`. Wikidata utilise des identifiants numeriques (Q-items, P-properties) plutot que des URI lisibles.

Cherchons les langages de programmation crees apres 2010.

In [18]:
# Query Wikidata for programming languages created after 2010
sparql_wd = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql_wd.setReturnFormat(JSON)

# Add a User-Agent header (required by Wikidata policy)
sparql_wd.agent = "CoursIA-SemanticWeb-Notebook/1.0 (educational)"

sparql_wd.setQuery("""
SELECT ?lang ?langLabel ?inception
WHERE {
    ?lang wdt:P31 wd:Q9143 .        # instance of: programming language
    ?lang wdt:P571 ?inception .      # inception date
    FILTER (YEAR(?inception) >= 2010)
    SERVICE wikibase:label { bd:serviceParam wikibase:language "fr,en" . }
}
ORDER BY DESC(?inception)
LIMIT 10
""")

try:
    results_wd = sparql_wd.query().convert()
    bindings_wd = results_wd["results"]["bindings"]

    print(f"=== Langages de programmation recents (Wikidata) ===")
    print(f"{'Langage':<25s} {'Annee de creation':>20s}")
    print("-" * 45)
    for r in bindings_wd:
        name = r["langLabel"]["value"]
        year = r["inception"]["value"][:4]  # extract year from date
        print(f"{name:<25s} {year:>20s}")

except Exception as e:
    print(f"Erreur lors de la requete Wikidata : {type(e).__name__}: {e}")
    print("L'endpoint Wikidata est peut-etre temporairement indisponible.")
    print("Cela n'empeche pas de continuer le notebook.")

=== Langages de programmation recents (Wikidata) ===
Langage                      Annee de creation
---------------------------------------------
Varphi Language                           2025
Pkl                                       2024
Jakt                                      2022
Delegua                                   2022
Mavka                                     2022
Carbon                                    2020
BQN                                       2020
V                                         2019
Bosque                                    2019
Project Verona                            2019


### Interpretation : Requete Wikidata

**Differences avec DBpedia** :

| Aspect | DBpedia | Wikidata |
|--------|---------|----------|
| **Identifiants** | URIs lisibles (`dbr:Paris`) | Q-items (`wd:Q90` = Paris) |
| **Proprietes** | Ontologie riche (`dbo:populationTotal`) | P-properties (`wdt:P1082`) |
| **Labels** | `rdfs:label` + `FILTER(lang())` | `SERVICE wikibase:label` |
| **User-Agent** | Optionnel | **Obligatoire** (politique Wikimedia) |
| **Fraicheur** | Mises a jour periodiques | Mises a jour en temps reel |

> **Note** : `SERVICE wikibase:label` est un mecanisme specifique a Wikidata qui resout automatiquement les labels dans la langue demandee. C'est plus pratique que de filtrer manuellement par langue.

---

## 6. Comparaison dotNetRDF (.NET) vs rdflib (Python)

Ce tableau recapitule les equivalences entre les deux bibliotheques que nous utilisons dans cette serie. Il vous permettra de traduire facilement du code d'un langage a l'autre.

| Operation | dotNetRDF (C#) | rdflib (Python) |
|-----------|---------------|----------------|
| **Creer un graphe** | `var g = new Graph();` | `g = Graph()` |
| **Noeud URI** | `g.CreateUriNode(new Uri("..."))` | `URIRef("...")` |
| **Blank node** | `g.CreateBlankNode()` | `BNode()` |
| **Litteral simple** | `g.CreateLiteralNode("text")` | `Literal("text")` |
| **Litteral type** | `g.CreateLiteralNode("42", XSD.Integer)` | `Literal(42, datatype=XSD.integer)` |
| **Litteral avec langue** | `g.CreateLiteralNode("Bonjour", "fr")` | `Literal("Bonjour", lang="fr")` |
| **Ajouter un triple** | `g.Assert(new Triple(s, p, o))` | `g.add((s, p, o))` |
| **Retirer un triple** | `g.Retract(new Triple(s, p, o))` | `g.remove((s, p, o))` |
| **Nombre de triples** | `g.Triples.Count()` | `len(g)` |
| **Serialiser en Turtle** | `new CompressingTurtleWriter().Save(g, file)` | `g.serialize(format="turtle")` |
| **Serialiser en RDF/XML** | `new RdfXmlWriter().Save(g, file)` | `g.serialize(format="xml")` |
| **Charger un fichier** | `FileLoader.Load(g, "file.ttl")` | `g.parse("file.ttl")` |
| **Requete SPARQL** | `g.ExecuteQuery("SELECT...")` | `g.query("SELECT...")` |
| **Endpoint distant** | `new SparqlRemoteEndpoint(uri)` | `SPARQLWrapper(uri)` |
| **Namespace** | `g.NamespaceMap.AddNamespace("ex", uri)` | `g.bind("ex", Namespace(uri))` |

### Differences philosophiques

| Aspect | dotNetRDF | rdflib |
|--------|-----------|--------|
| **Style** | OOP verbeux, typage fort | Pythonique, concis |
| **Noeuds** | Lies au graphe (`g.CreateUriNode`) | Independants (`URIRef(...)`) |
| **Triples** | Objet `Triple(s, p, o)` | Tuple Python `(s, p, o)` |
| **SPARQL** | Retourne `SparqlResultSet` | Retourne `Result` iterable |
| **Extensibilite** | Plugins NuGet | Plugins PyPI (pyshacl, owlready2...) |

> **A retenir** : Les deux bibliotheques implementent les memes standards W3C. Le choix depend de votre ecosysteme : .NET pour l'integration avec des applications C#, Python pour le data science et l'IA.

---

## Exercices

Mettez en pratique les concepts appris dans ce notebook avec ces trois exercices.

### Exercice 1 : Reproduire le graphe "Hello World" de SW-2

Dans le notebook SW-2 (dotNetRDF), nous avons cree un graphe contenant :
```
<http://www.dotnetrdf.org>  <http://example.org/says>  "Hello World"
<http://www.dotnetrdf.org>  <http://example.org/says>  "Bonjour tout le Monde"@fr
```

Reproduisez ce graphe en Python avec rdflib, puis serialisez-le en Turtle et en N-Triples.

**Indice** : Utilisez `URIRef` pour les sujets/predicats et `Literal` avec `lang="fr"` pour le texte francais.

In [19]:
# Exercice 1 : Reproduire le graphe Hello World de SW-2
# Completez le code ci-dessous

g_hello = Graph()

# Define subject and predicate URIs
dotnetrdf = URIRef("http://www.dotnetrdf.org")
says = URIRef("http://example.org/says")

# TODO: Add the two triples
# g_hello.add((..., ..., ...))
# g_hello.add((..., ..., ...))

# Verify
print(f"Nombre de triples : {len(g_hello)} (attendu: 2)")
print()
print("=== Turtle ===")
print(g_hello.serialize(format="turtle"))
print("=== N-Triples ===")
print(g_hello.serialize(format="nt"))

Nombre de triples : 0 (attendu: 2)

=== Turtle ===


=== N-Triples ===



### Exercice 2 : Interroger animals.ttl pour trouver les mammiferes

Ecrivez une requete SPARQL qui retourne tous les animaux qui sont des mammiferes (instances de `ex:Mammal` ou de ses sous-classes `ex:Dog`, `ex:Cat`), avec leur nom et leur age.

**Indice** : Vous pouvez utiliser `rdfs:subClassOf` pour remonter la hierarchie, ou lister les classes directement avec `UNION`.

In [20]:
# Exercice 2 : Trouver tous les mammiferes dans animals.ttl
# g_animals est deja charge depuis la section 4

query_mammals = """
PREFIX ex: <http://example.org/animals#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?name ?age ?type_label
WHERE {
    # TODO: Completez la requete
    # Trouvez les animaux dont le type est une sous-classe de ex:Mammal
    # (ou ex:Mammal lui-meme)
}
ORDER BY ?name
"""

# results = g_animals.query(query_mammals)
# for row in results:
#     print(f"{row.name} ({row.type_label}) - {row.age} ans")

### Exercice 3 : Requete Wikidata pour les villes francaises

Utilisez SPARQLWrapper pour interroger Wikidata et obtenir les 10 villes de France les plus peuplees, avec leur nom et leur population.

**Indices** :
- Classe ville : `wd:Q515` (city)
- Pays : `wdt:P17` (country), France = `wd:Q142`
- Population : `wdt:P1082`
- Pensez au `try/except` et au `User-Agent`

In [21]:
# Exercice 3 : Top 10 villes francaises via Wikidata

sparql_ex3 = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql_ex3.setReturnFormat(JSON)
sparql_ex3.agent = "CoursIA-SemanticWeb-Notebook/1.0 (educational)"

sparql_ex3.setQuery("""
# TODO: Completez la requete SPARQL
SELECT ?cityLabel ?population
WHERE {
    # instance of: city
    # country: France
    # population
    # SERVICE wikibase:label
}
ORDER BY DESC(?population)
LIMIT 10
""")

try:
    results_ex3 = sparql_ex3.query().convert()
    for r in results_ex3["results"]["bindings"]:
        print(f"{r['cityLabel']['value']} : {int(r['population']['value']):,d} habitants")
except Exception as e:
    print(f"Erreur : {e}")
    print("Verifiez la syntaxe de votre requete ou la disponibilite de l'endpoint.")

Erreur : 'cityLabel'
Verifiez la syntaxe de votre requete ou la disponibilite de l'endpoint.


---

## Resume

Ce notebook a presente l'ecosysteme Python pour le Web Semantique, centre sur **rdflib** et **SPARQLWrapper**.

### Concepts cles

| Concept | Ce que vous avez appris |
|---------|------------------------|
| **rdflib** | Creer des graphes, ajouter des triples, gerer les namespaces |
| **Types de noeuds** | URIRef, BNode, Literal (avec datatype et lang) |
| **Serialisation** | Turtle, N-Triples, RDF/XML, JSON-LD |
| **SPARQL local** | `g.query()` avec SELECT, FILTER, OPTIONAL |
| **SPARQLWrapper** | Interroger DBpedia et Wikidata |
| **Comparaison** | Equivalences dotNetRDF / rdflib |

### Prochaines etapes

Dans le notebook suivant (**SW-9 : SHACL**), nous utiliserons **pySHACL** pour valider la qualite de nos donnees RDF contre des formes (shapes) qui definissent des contraintes structurelles.

---

**Navigation** : [<< 7-OWL](SW-7-OWL.ipynb) | [Index](README.md) | [9-SHACL >>](SW-9-SHACL.ipynb)